# Backtesting

Back testing is basically lingo for testing the performance of a strategy (or a prediction model) on historical data. The trader would need to consider various things before back testing especially with regards to the historical data.

## Using Historical Data

There are many free and commercial historical data that can be accessed. We will focus mostly on google's historical data and talk about issues such as data splitting and dividend adjustment

### Data Splitting

Companies may have their stock split N to 1. So previous a stock would have been \$140 per share on Aug 1 2015 and they may have decided to split it 2 to 1 thus the price that shows up on Yahoo would be $70 per share on Aug 2 2015. In such a case if we analyze the data without adjusting for splitting then the algorithm may think that there was a decrease in that time... which is false. So we need to adjust the previous stock price accordingly. Because the split happend as a 2 to 1, we may divide the previous values by 2 but this isn't the only adjustment that needs to be made. All dividens also needs to be accounted for. Let's generate some data below.

In [47]:
import pandas as pd
import numpy as np

In [50]:
np.random.seed(0)

# generate fake data
date_range = pd.Series(pd.date_range('2015-08-01', periods=10))
df = pd.DataFrame({
    'date': pd.concat([date_range]),
    'close': np.concatenate((np.random.normal(140, 1, 5),np.random.normal(70, 2, 5)),axis = 0)
}).reset_index(drop=True)
print(df)

        close       date
0  141.764052 2015-08-01
1  140.400157 2015-08-02
2  140.978738 2015-08-03
3  142.240893 2015-08-04
4  141.867558 2015-08-05
5   68.045444 2015-08-06
6   71.900177 2015-08-07
7   69.697286 2015-08-08
8   69.793562 2015-08-09
9   70.821197 2015-08-10


As we can see in the dataframe on Aug 6th the stock split and we would need to get the dividend information in order to adjust according to the dividends earned after Aug 5th. Here is a random generation of dividen data

In [65]:
date_range = pd.Series(pd.date_range('2015-08-01', '2017-08-01'))

div = pd.DataFrame({
    'date': np.random.choice(date_range[1:-1], size = 10, replace=False),
    'prevCLose' : np.random.normal(70, 2, 10),
    'dividend': np.random.beta(10,10,10)
}).reset_index(drop=True)
print(div)

        date  dividend  prevCLose
0 2015-12-28  0.437752  68.936788
1 2016-07-19  0.447843  69.175456
2 2015-09-04  0.496649  69.193082
3 2015-10-30  0.515120  66.339943
4 2015-11-02  0.295963  68.608330
5 2017-03-13  0.576377  70.493532
6 2016-04-15  0.734996  73.051915
7 2017-07-03  0.542516  68.454456
8 2017-06-22  0.675151  71.764113
9 2017-03-03  0.441837  67.494813


Based on the dividend information we need to calculate the multiplier information as follows:
$$ \text{Multiplier} = \frac{\text{Previous Closing Price} - \text{Dividend}}{\text{Previous Closing Price}} $$

In [67]:
div['Multiplier'] = (div['prevCLose'] - div['dividend'])/div['prevCLose']

In [68]:
print(div)

        date  dividend  prevCLose  Multiplier
0 2015-12-28  0.437752  68.936788    0.993650
1 2016-07-19  0.447843  69.175456    0.993526
2 2015-09-04  0.496649  69.193082    0.992822
3 2015-10-30  0.515120  66.339943    0.992235
4 2015-11-02  0.295963  68.608330    0.995686
5 2017-03-13  0.576377  70.493532    0.991824
6 2016-04-15  0.734996  73.051915    0.989939
7 2017-07-03  0.542516  68.454456    0.992075
8 2017-06-22  0.675151  71.764113    0.990592
9 2017-03-03  0.441837  67.494813    0.993454


In [71]:
np.prod(div['Multiplier'])*0.5

0.46411004688410229

The product of the multipliers after the split date would need to be used as the adjustment for the previous time points before stock splitting.

In [198]:
for x in range(len(df)):
    if df.loc[x, 'date'] < pd.datetime(2015,8,6):
        df.loc[x,'adj_close'] = df.loc[x,'close']*np.prod(div['Multiplier'])*0.5
    else:
        df.loc[x,'adj_close'] = df.loc[x,'close']

df

,close,date,adj_close
0,141.764052,2015-08-01,65.794121
1,140.400157,2015-08-02,65.161124
2,140.978738,2015-08-03,65.429649
3,142.240893,2015-08-04,66.015428
4,141.867558,2015-08-05,65.842159
5,68.045444,2015-08-06,68.045444
6,71.900177,2015-08-07,71.900177
7,69.697286,2015-08-08,69.697286
8,69.793562,2015-08-09,69.793562
9,70.821197,2015-08-10,70.821197


Thus as we can see the adjusted prices pre Aug 06 will be adjusted accordingly and then the algorithm should be run on this data set.

## Survivor ship bias in data

Since data that contains survivorship of companies (i.e. companies that have gone backrupt or delisted are expensive, we need to either 1. collect current data to make sure we include survivorship of companies from here on out or 2. make lots of money and buy the proprietary data that includes such information.

One thing to be cognizant is the effect of survivorship bias. If we focus on getting low priced stocks, it is likely that those companies are going bankrupt soon... so read into not just the price but other qualitative information (news and etc) to determine if that is the case and stay away from those stocks.

## High and Low Data

the high and low prices are far more noisier compared to the open and close prices. There are many possibilities that may cause the reliability of the high and low price to be worse than that of the open and close prices. FOr example, a order may not have been filled due to the fact that a very small order was transacted at the high, or the execution could have occured ona a market to which the order wasn't routed.

Basically, the discrepancies of the open and close prices usually have less impact on backtest performance than errors in the high and low prices, since high and low price errors almost always inflate the backtest returns. 

Moral of this section is to do at least a simple error check of the dataset with high, low, open and closeing prices such that various combinations of daily returns can be calculated.

## Performance Measures

There are various performance measures that people may use and we will not go over all of them. Most of the time return based measures will require explanation regarding the denominator and numerator. Thus it is relatively straight forward to use the Sharp ratio and drawdown values.

In terms of using the Sharp ratio, the risk free rate from the returns of a dollar-neutral portfolio (i.e. long short equal capital) should not be subtracted. The reason is because a dollar-neutral portfolio is self-financing (i.e. the cash earned from selling short pays for the purchase of long securities so we assume that the financing cost is small and can be neglected for backtesting). Meanwhile, the margin balance that needs to be maintained earns a credit interest close to the risk-free rate $r_F$ thus the excess return used in calculating the Sharp Ratio would be the strategy return $R$ plus the return from the self-financing $r_F$ subtratced by the credit interested $r_F$ would be $R + r_F - r_F = R$. 
In general, only subtracte the risk-free rate from the $R$ when calculating the Sharp ratio if there are financing costs that are involved specifically for long-only strategies.

Another aspect to facilitate comparison across strategies is to annualize the Sharp ratio. If we are using monthly returns, the average annual return is 12 * the monthly return. The standard deviation would be $\sqrt{12}$ according to Sharpe. Here is the general equation:
$$ \text{Annualized Sharpe Ratio} = \sqrt{N_T} \times \text{Sharpe Ratio Based on T}$$
Where T is the trading period.

For example if we trade only for 6.5 hours a day for 252 days then $N_T = 252 \times 6.5 = 1638$ so the Annulized Sharpe Ratio would be $\sqrt{1638} \times R/s$.

Further calculation and example using bitcoin is in [Definition and Formulas](https://github.com/ck2136/QuantTrade/blob/master/Definitions%20and%20Formulas.ipynb) Notebook.

## COmmon Backtesting Pitfalls to Avoid

We've already talked about how survivorship bias can inflate strategy performance. There are two other common mistakes to avoid.

### Look-Ahead Bias

If our strategy for buying involves buying at 1% of the day's low, this introduces the *look-ahead bias*. Basically we wouldn't know what the day's low until we had all values when the market closes. The way to avoid this is to use *lagged* historical data for calculating signals at every opportunity. This basically translates into conducting analyses with data up to the close of the *previous* trading period only. In `R` or `Python` this would involve using a lag function to run aanalses on a series of columns that would produce a *lagged* column for regression.

Programmatically to check for look-ahead bias, we would create two datasets where one is using the original data and running the regression then truncating the most recent N observations while a new data set would also have the most recent N data removed but only after truncation through *lagging* do we run the regression. We then align the two predicted positions based on the regression and see if there were any difference. If there are then there is look-ahead bias. 

### Data-snooping bias

This has to do with over fitting the data with numerous parameters and going back in time more than 10 years than may not be relevant. The goal is to get a parsimonimous model that really only involves parameters that will be highly predictive of future positions. The author of QT indicates that using aroudn 5 parameters given highly dependent dataset would normally suffice in creating moving-average models.

In order to mitigate data snooping bias there are some considerations below.

#### Sample Size

The greater then number of parameter used, the more data points required for optimizing the model. The statistical community is in constant development regarding the optimal sample size requirement for time-series prediction models. Check [Hanke and Wichern](https://www.amazon.com/dp/0132301202/?tag=stackoverflow17-20)

#### Cross Validation

In order to robustly check whether or not the predictions are really going to be useful, there needs to be an out-of-sample testing dataset that can be used to test the prediction model.

More robust optimization of the model would employ moving optimizaiton of the parameters. The parameters are constantly adapting to the changing historical data and data snooping in that regard would be reduced.

#### Sensitivity Analysis

Once optimization has been achieved using the various ideas described above, varying the conditions to see how the results of the prediction model changes on the training and testing data. If there is considerable difference in the performance masures such that only the optimal condition is apporpriate, that means that the strategy may not be robust therefore may result in poor predictions down the line. Important variations that need to tried:
- Simplify the model
    - Eliminate conditions, constraints, and parameters to make the model more parsimonious
- Set up trading strategy such that multiple variation of the final model (which has been tested to have good performance) will be used such that trading capital should be divided among them

### Transaction Costs

Also one should take into consideration the transaction cost when thinking about developing indicators for buying and selling. This will again be implemented in the other notebook.

### Strategy Refinement

- Pair trading: Buying and selling stocks from similar stock universe
- Exclusion of stocks or groups of stocks from universe